# Calendar View

Given a department code and a term, creates a calendar view of scheduled courses in Banner for the purpose of visualizing course scheduling conflicts.

The primary use case for this is determining where to schedule a new IIS class so as to minimize conflicts with other departments, but it would probably be equally useful for a department to visualize its own schedule.

Note that you will need to install and reference NuGet packages as appropriate on your system.

## Basic Banner Query and HTML walking

The code below describes a basic Banner POST for all courses offered by a department as well as some methods for processing the html

In [53]:
#r "/z/aolney/repos/FSharp.Data.2.3.2/lib/net40/FSharp.Data.dll"
#r "/z/aolney/repos/HtmlAgilityPack.1.4.9.5/lib/Net40/HtmlAgilityPack.dll"
open FSharp.Data
open HtmlAgilityPack
open System

let QueryDepartment (dept:string) (term:string) =
    let result = Http.RequestString( 
        "https://banssbprod.memphis.edu/pls/PROD/bwckschd.p_get_crse_unsec",
        query=[
            "term_in", term.Trim();
            "sel_subj","dummy";
            "sel_day","dummy";
            "sel_schd","dummy";
            "sel_insm","dummy";
            "sel_camp","dummy";
            "sel_levl","dummy";
            "sel_sess","dummy";
            "sel_instr","dummy";
            "sel_ptrm","dummy";
            "sel_attr","dummy";
            "sel_subj",dept.Trim();
            "sel_crse",""; //number.Trim();
            "sel_title","";
            "sel_insm","%";
            "sel_from_cred","";
            "sel_to_cred","";
            "sel_camp","%";
            "sel_levl","%";
            "sel_ptrm","%";
            "sel_instr","%";
            "sel_attr","%";
            "begin_hh","0";
            "begin_mi","0";
            "begin_ap","a";
            "end_hh","0";
            "end_mi","0";
            "end_ap","a" ], 
        httpMethod="POST")
    //
    result

//Creates a doc from html string
let createDoc html =
    let doc = new HtmlDocument()
    doc.LoadHtml html
    doc.DocumentNode

//A simple version of the more complex https://stackoverflow.com/questions/22661640/how-to-fix-ill-formed-html-with-html-agility-pack
//However, it appears htmlagilitypack does not detect errors very well
let createDocAutoFix html =
    let doc = new HtmlDocument()
    doc.LoadHtml html
    let tempHtml = html.Split( [|"\n";"\r\n"|], StringSplitOptions.None ) //we need line numbers
    if doc.ParseErrors <> null then
        for error in doc.ParseErrors do
            if error.Code = HtmlParseErrorCode.TagNotOpened || error.Code = HtmlParseErrorCode.TagNotClosed then
                tempHtml.[error.Line] <- tempHtml.[error.Line].Remove(error.LinePosition, error.SourceText.Length)
    doc.LoadHtml (tempHtml |> String.concat "\n")
    doc.DocumentNode
    
let descendants (name : string) (node : HtmlNode) = 
    node.Descendants name
    
let selectNodes (query : string) (node : HtmlNode) = 
    node.SelectNodes query

let inline innerText (node : HtmlNode) = 
    node.InnerText

let inline attr name (node : HtmlNode) = 
    node.GetAttributeValue(name, "")

let inline hasAttr name value node = 
    attr name node = value

Possible incorrect indentation: this token is offside of context started at position (8:18). Try indenting this token further or using standard formatting conventions.
Possible incorrect indentation: this token is offside of context started at position (8:18). Try indenting this token further or using standard formatting conventions.
Possible incorrect indentation: this token is offside of context started at position (8:18). Try indenting this token further or using standard formatting conventions.
Possible incorrect indentation: this token is offside of context started at position (8:18). Try indenting this token further or using standard formatting conventions.

## Calendar View

Run the query to get all courses in a department, extract the time/date information for each course, then export this information in ICS format.

In [57]:
//Banner Term, e.g. 201680 for Fall 2016; 201610 for Spring
let term = "201910"
let dept = "COMP"

type Class =
    {
        Title: string
        Type: string
        Time: string
        StartDateTime: DateTime //hackish: date and time components have slightly different semantics
        EndDateTime: DateTime //hackish: date and time components have slightly different semantics
        Days: string[]
        Where: string
        DateRange: string
        ScheduleType: string
        Instructors: string
    }

let GetVEvents (c : Class) =
    let s = c.StartDateTime
    let e = c.EndDateTime
    let dayEndTime = new DateTime(s.Year,s.Month,s.Day,e.Hour,e.Minute,e.Second)
    seq {
        for day in c.Days do
            yield """BEGIN:VEVENT
DTSTART;TZID=America/Chicago:#START#
DTEND;TZID=America/Chicago:#END#
RRULE:FREQ=WEEKLY;UNTIL=#UNTILZ#;BYDAY=#DAY#
DTSTAMP:#NOWZ#
CREATED:#NOWZ#
DESCRIPTION:
LAST-MODIFIED:#NOWZ#
LOCATION:
SEQUENCE:0
STATUS:CONFIRMED
SUMMARY:#NAME#
TRANSP:OPAQUE
END:VEVENT""".Replace("#START#",c.StartDateTime.ToString("yyyyMMddTHHmmss")).Replace("#END#",dayEndTime.ToString("yyyyMMddTHHmmss")).Replace("#UNTILZ#",c.EndDateTime.ToUniversalTime().ToString("yyyyMMddTHHmmssZ")).Replace("#DAY#",day).Replace("#NOWZ#",DateTime.Now.ToUniversalTime().ToString("yyyyMMddTHHmmssZ")).Replace("#NAME#",c.Title)
    }

let GetDays dayString = 
    if dayString = "&nbsp;" then
        [||]
    else 
        //originally used System.DayOfWeek, but storing as ical is more useful
        let result = new ResizeArray<string>()
        for c in dayString do
            match c with
            | 'M' -> result.Add( "MO" )//DayOfWeek.Monday )
            | 'T' -> result.Add( "TU" ) //DayOfWeek.Tuesday )
            | 'W' -> result.Add( "WE" ) //DayOfWeek.Wednesday )
            | 'R' -> result.Add( "TH" ) //DayOfWeek.Thursday )
            | 'F' -> result.Add( "FR" ) //DayOfWeek.Friday )
            | 'S' -> result.Add( "SA" ) //DayOfWeek.Saturday )
            | _ -> ()
        //
        result.ToArray()
        
let GetStartEndDateTime (dateNode:HtmlNode) (timeNode:HtmlNode) = 
    let dateSplit = dateNode.InnerHtml.Split('-')
    let timeSplit = timeNode.InnerHtml.Split('-')
    match dateSplit.Length,timeSplit.Length with
    //we have valid dates and times
    | 2,2 -> 
        let s = dateSplit.[0].Trim() + " " + timeSplit.[0].Trim() |> DateTime.Parse
        let e = dateSplit.[1].Trim() + " " + timeSplit.[1].Trim() |> DateTime.Parse
        s,e
    //all other cases we can't put on calendar; we use MinValue instead of options
    | 2,_ -> DateTime.MinValue,DateTime.MinValue 
    | _,2 -> DateTime.MinValue,DateTime.MinValue 
    | _,_ -> DateTime.MinValue,DateTime.MinValue

    
let classEvents =
    QueryDepartment dept term
    |> createDoc
    |> selectNodes "html/body/div/table/tr/th/a | html/body/div/table/tr/td/table/tr/td"
    //|> Seq.map( fun x -> x.InnerHtml)
    |> Seq.chunkBySize 8 //first is title; next 7 is table of info
    //|> Seq.take 19 //for debug
    |> Seq.map( fun classChunk -> 
        let days = GetDays classChunk.[3].InnerHtml
        let start,stop = GetStartEndDateTime classChunk.[5] classChunk.[2]
        {
            Title=classChunk.[0].InnerHtml;
            Type=classChunk.[1].InnerHtml;
            Time=classChunk.[2].InnerHtml;
            StartDateTime =  start
            EndDateTime = stop
            Days=days;
            Where=classChunk.[4].InnerHtml;
            DateRange=classChunk.[5].InnerHtml;
            ScheduleType=classChunk.[6].InnerHtml;
            Instructors=classChunk.[7].InnerText;
            }
      )
    //Purge events without times
    |> Seq.filter (fun x -> x.StartDateTime <> DateTime.MinValue)
    |> Seq.collect GetVEvents
    |> Seq.toList

//Make ICS
// Looked at https://github.com/UHCalendarTeam/ANtICalendar but it wasn't good
// Instead made a calendar template on Google and reverse engineered it
let prefix = """BEGIN:VCALENDAR
PRODID:-//Google Inc//Google Calendar 70.9054//EN
VERSION:2.0
CALSCALE:GREGORIAN
METHOD:PUBLISH
X-WR-CALNAME:#NAME#
X-WR-TIMEZONE:America/Chicago
BEGIN:VTIMEZONE
TZID:America/Chicago
X-LIC-LOCATION:America/Chicago
BEGIN:DAYLIGHT
TZOFFSETFROM:-0600
TZOFFSETTO:-0500
TZNAME:CDT
DTSTART:19700308T020000
RRULE:FREQ=YEARLY;BYMONTH=3;BYDAY=2SU
END:DAYLIGHT
BEGIN:STANDARD
TZOFFSETFROM:-0500
TZOFFSETTO:-0600
TZNAME:CST
DTSTART:19701101T020000
RRULE:FREQ=YEARLY;BYMONTH=11;BYDAY=1SU
END:STANDARD
END:VTIMEZONE""".Replace("#NAME#",dept)


let icsString = prefix + "\n" + (classEvents |> String.concat "\n") + "\n" + "END:VCALENDAR"
System.IO.File.WriteAllText(dept + ".ics", icsString)
"DONE"

"DONE"